In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Geometry Parameters
R1 = 1.0  # Radius of the cone base
H1 = 2.0  # Height of the cone
R2 = 2.0  # Radius of the cylinder
H2 = 5.0  # Length of the cylinder
U_inf = 1.0  # Freestream velocity

# Panel Discretization Parameters
N_cylinder = 50  # Number of panels on the cylinder
N_cone = 50  # Number of panels along the cone height
N_theta = 50  # Angular resolution for cone

# Discretize the Cylinder Surface
z_cylinder = np.linspace(0, H2, N_cylinder)
theta_cylinder = np.linspace(0, 2 * np.pi, N_cylinder)
x_cylinder = R2 * np.cos(theta_cylinder)
y_cylinder = R2 * np.sin(theta_cylinder)
z_cylinder = np.tile(z_cylinder, len(theta_cylinder))

x_cylinder = np.repeat(x_cylinder, N_cylinder)
y_cylinder = np.repeat(y_cylinder, N_cylinder)

# Discretize the Cone Surface
z_cone = np.linspace(0, H1, N_cone)
theta_cone = np.linspace(0, 2 * np.pi, N_theta)
r_cone = R1 * (1 - z_cone / H1)  # Linear decrease in radius

# Mesh grid for cone
theta_cone_mesh, z_cone_mesh = np.meshgrid(theta_cone, z_cone)
r_cone_mesh = R1 * (1 - z_cone_mesh / H1)

x_cone = r_cone_mesh * np.cos(theta_cone_mesh)
y_cone = r_cone_mesh * np.sin(theta_cone_mesh)
z_cone = z_cone_mesh

# Combine Panel Points
x_panels = np.hstack([x_cylinder, x_cone.flatten()])
y_panels = np.hstack([y_cylinder, y_cone.flatten()])
z_panels = np.hstack([z_cylinder, z_cone.flatten()])

# Number of Panels
N_panels = len(x_panels)

# Influence Coefficient Matrix
A = np.zeros((N_panels, N_panels))
b = np.zeros(N_panels)

# Boundary Conditions and Velocity Influences
for i in range(N_panels):
    for j in range(N_panels):
        # Distance between control point and source panel
        dx = x_panels[i] - x_panels[j]
        dy = y_panels[i] - y_panels[j]
        dz = z_panels[i] - z_panels[j]
        r = np.sqrt(dx**2 + dy**2 + dz**2)
        if r > 1e-6:  # Avoid singularities
            A[i, j] = 1 / (4 * np.pi * r)  # Influence coefficient
        
    # Freestream contribution to boundary condition
    b[i] = -U_inf * z_panels[i] / np.sqrt(x_panels[i]**2 + y_panels[i]**2 + z_panels[i]**2)

# Solve for Source Strengths
sigma = np.linalg.solve(A, b)

# Calculate Velocity Field
x_grid, y_grid, z_grid = np.meshgrid(
    np.linspace(-R2, R2, 20),
    np.linspace(-R2, R2, 20),
    np.linspace(0, H2, 20)
)
u_field = np.zeros_like(x_grid)
v_field = np.zeros_like(y_grid)
w_field = np.zeros_like(z_grid)

for i in range(x_grid.shape[0]):
    for j in range(x_grid.shape[1]):
        for k in range(x_grid.shape[2]):
            for l in range(N_panels):
                dx = x_grid[i, j, k] - x_panels[l]
                dy = y_grid[i, j, k] - y_panels[l]
                dz = z_grid[i, j, k] - z_panels[l]
                r = np.sqrt(dx**2 + dy**2 + dz**2)
                if r > 1e-6:
                    u_field[i, j, k] += sigma[l] * dx / (4 * np.pi * r**3)
                    v_field[i, j, k] += sigma[l] * dy / (4 * np.pi * r**3)
                    w_field[i, j, k] += sigma[l] * dz / (4 * np.pi * r**3)

# Plot the Velocity Field
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(projection='3d')
ax.quiver(x_grid, y_grid, z_grid, u_field, v_field, w_field, length=0.1, normalize=True)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Velocity Field Around Turbofan Inlet')
plt.show()

LinAlgError: Singular matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Geometry Parameters
R2 = 2.0  # Radius of the cylinder cross-section
L_cone = 4.0  # Length of the cone cross-section
U_inf = 1.0  # Freestream velocity
N_panels = 100  # Number of panels for both cylinder and cone

# Discretize the Cylinder (Circle)
theta = np.linspace(0, 2 * np.pi, N_panels // 2, endpoint=False)
x_cylinder = R2 * np.cos(theta)
y_cylinder = R2 * np.sin(theta)

# Discretize the Cone (Line Segment)
x_cone = np.linspace(-R2, 0, N_panels // 2)
y_cone = np.linspace(0, -L_cone, N_panels // 2)

# Combine Cylinder and Cone
x_panels = np.hstack([x_cylinder, x_cone])
y_panels = np.hstack([y_cylinder, y_cone])

# Panel Midpoints (Control Points)
x_mid = 0.5 * (x_panels[:-1] + x_panels[1:])
y_mid = 0.5 * (y_panels[:-1] + y_panels[1:])

# Panel Normals (Unit Vectors)
dx = np.diff(x_panels)
dy = np.diff(y_panels)
lengths = np.sqrt(dx**2 + dy**2)
nx = dy / lengths  # x-component of normal
ny = -dx / lengths  # y-component of normal

# Number of Panels
N_panels = len(x_mid)

# Influence Coefficient Matrix
A = np.zeros((N_panels, N_panels))
b = np.zeros(N_panels)

for i in range(N_panels):
    for j in range(N_panels):
        if i == j:
            A[i, j] = 0.5  # Self-influence of a panel
        else:
            dx_p = x_mid[i] - x_panels[j]
            dy_p = y_mid[i] - y_panels[j]
            r = np.sqrt(dx_p**2 + dy_p**2)
            A[i, j] = (dx_p * nx[j] + dy_p * ny[j]) / (2 * np.pi * r**2)  # Influence of panel j on i
    
    # Freestream contribution to the RHS
    b[i] = -U_inf * (nx[i] * 1 + ny[i] * 0)  # Freestream aligned with x-direction

# Solve for Source Strengths
sigma = np.linalg.solve(A, b)

# Calculate Velocity at Midpoints
V_tangential = np.zeros(N_panels)
for i in range(N_panels):
    for j in range(N_panels):
        if i != j:
            dx_p = x_mid[i] - x_panels[j]
            dy_p = y_mid[i] - y_panels[j]
            r = np.sqrt(dx_p**2 + dy_p**2)
            V_tangential[i] += sigma[j] / (2 * np.pi * r)

    V_tangential[i] += U_inf * np.cos(np.arctan2(ny[i], nx[i]))

# Calculate Pressure Coefficient
Cp = 1 - (V_tangential / U_inf)**2

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(x_mid, Cp, '-o', label='Pressure Coefficient')
plt.gca().invert_yaxis()
plt.title('Pressure Coefficient Distribution')
plt.xlabel('x')
plt.ylabel('$C_p$')
plt.grid()
plt.legend()
plt.show()

# Plot Geometry
plt.figure(figsize=(10, 6))
plt.plot(x_panels, y_panels, 'k-', label='Turbofan Inlet Geometry')
plt.quiver(x_mid, y_mid, nx, ny, scale=10, color='r', label='Panel Normals')
plt.axis('equal')
plt.title('Turbofan Inlet Geometry with Panel Normals')
plt.xlabel('x')
plt.ylabel('y')
plt.grid()
plt.legend()
plt.show()